<a href="https://colab.research.google.com/github/paulo-lins/Sales-Analysis/blob/main/sale_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
 

In [3]:
df = pd.read_csv("/content/sample_data/all_data.csv")

In [4]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"


In [5]:
df[df['Order ID'].duplicated(keep=False)]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
17,176574,Google Phone,1,600,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001"
18,176574,USB-C Charging Cable,1,11.95,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001"
29,176585,Bose SoundSport Headphones,1,99.99,04/07/19 11:31,"823 Highland St, Boston, MA 02215"
...,...,...,...,...,...,...
95459,213546,USB-C Charging Cable,1,11.95,06/19/19 13:04,"822 11th St, Atlanta, GA 30301"
95467,213554,iPhone,1,700,06/02/19 00:38,"890 Jefferson St, San Francisco, CA 94016"
95468,213554,Flatscreen TV,1,300,06/02/19 00:38,"890 Jefferson St, San Francisco, CA 94016"
95473,213559,Google Phone,1,600,06/10/19 12:14,"621 Cedar St, Los Angeles, CA 90001"


In [6]:
pd.__version__

'1.3.5'